<a href="https://colab.research.google.com/github/markuskunej/air-pollution-thesis/blob/master/VARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
DATA_PATH = "/content/Y&E_60m_last40days"
!unzip "/content/Y&E_60m_last40days.zip"

Archive:  /content/Y&E_60m_last40days.zip
  inflating: Y&E_60m_last40days/AQI.csv  
  inflating: Y&E_60m_last40days/CO.csv  
  inflating: Y&E_60m_last40days/CO2.csv  
  inflating: Y&E_60m_last40days/Elevation.csv  
  inflating: Y&E_60m_last40days/Humidity.csv  
  inflating: Y&E_60m_last40days/Latitude.csv  
  inflating: Y&E_60m_last40days/Longitude.csv  
  inflating: Y&E_60m_last40days/NO.csv  
  inflating: Y&E_60m_last40days/NO2.csv  
  inflating: Y&E_60m_last40days/Noise_LEQ.csv  
  inflating: Y&E_60m_last40days/Noise_Max.csv  
  inflating: Y&E_60m_last40days/O3.csv  
  inflating: Y&E_60m_last40days/PM1.csv  
  inflating: Y&E_60m_last40days/PM10.csv  
  inflating: Y&E_60m_last40days/PM2.5.csv  
  inflating: Y&E_60m_last40days/Pressure.csv  
  inflating: Y&E_60m_last40days/Temperature.csv  
  inflating: Y&E_60m_last40days/Wind_Direction.csv  
  inflating: Y&E_60m_last40days/Wind_Gust.csv  
  inflating: Y&E_60m_last40days/Wind_Speed.csv  


In [66]:
import pandas as pd
import os
import glob

DATA_PATH = "/content/Y&E_60m_last40days"
print(os.path.join(DATA_PATH , "/*.csv"))
all_files = glob.glob('/content/Y&E_60m_last40days/*.csv')
keys = []
dfs = []
big_df = []
for file_name in all_files:
  df = pd.read_csv(file_name, parse_dates=["Time"])
  variable_name = os.path.basename(file_name).split(".")[0]
  df[variable_name] = df.mean(axis=1)
  #keys.append(variable_name)
  #df.set_index(pd.to_datetime(df['Time'].astype(str), format='%Y-%m-%d %H:%M:%S'), inplace=True)
  # reduce dataframe columns to only the average
  df = df[["Time", variable_name]]

  print(df.index)
  dfs.append(df)

# concat dataframes into one
big_df = pd.concat(dfs, axis=1)
print(big_df.shape)
print(big_df.head())

/*.csv
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
RangeIndex(start=0, stop=962, step=1)
(962, 40)
                 Time  Pressure                Time  Wind_Speed  \
0 2022-10-09 11:00:00     999.1 2022-10-09 12:00:00      1.7820   
1 2022-10-09 12:00:00     999.4 2022-10-09 13:00:00      1.5498   
2 2022-10-09 13:00:00 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  
